# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
dataset = pd.read_excel('IDataset1.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_type        546 non-null    object 
 1   Charge_size        546 non-null    float64
 2   Standoff_distance  546 non-null    float64
 3   Impulse            546 non-null    float64
dtypes: float64(3), object(1)
memory usage: 17.2+ KB


In [3]:
# convert categorical variable into dummy variables
dataset = pd.get_dummies(dataset, columns=['Charge_type'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_size        546 non-null    float64
 1   Standoff_distance  546 non-null    float64
 2   Impulse            546 non-null    float64
 3   Charge_type_CompB  546 non-null    uint8  
 4   Charge_type_TNT    546 non-null    uint8  
dtypes: float64(3), uint8(2)
memory usage: 14.0 KB


In [4]:
dataset.head()

,Charge_size,Standoff_distance,Impulse,Charge_type_CompB,Charge_type_TNT
0,0.5,1.5,58.168617,0,1
1,0.5,2.5,38.798874,0,1
2,0.5,3.5,28.473915,0,1
3,0.5,4.5,22.343452,0,1
4,0.5,5.5,18.326342,0,1


In [5]:
y = dataset['Impulse']
X = dataset.drop('Impulse', axis=1)
print(X.shape, y.shape)

(546, 4) (546,)


In [6]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [8]:
# Function to create model, required for KerasClassifier
def create_model(optimizer):
 # create model
 model = Sequential()
 model.add(Dense(units=200, input_shape=(X_train.shape[1],), activation='relu'))
 model.add(Dense(units=100, activation='relu'))
 model.add(Dense(units=50, activation='relu'))
 model.add(Dense(units=1, activation='linear'))
 # Compile model
 model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['mae'])
 return model

In [9]:
# create model
model = KerasRegressor(model=create_model, verbose=0)

# define the grid search parameters
batch_size = [40, 50, 60, 70]
epochs = [200, 400, 450, 500]
optimizer = ['RMSprop', 'Adagrad', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.01, 0.1, 0.5, 1]

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs, model__optimizer=optimizer, optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid,  n_jobs=-1, cv=3, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.975733 using {'batch_size': 50, 'epochs': 400, 'model__optimizer': 'RMSprop', 'optimizer__learning_rate': 0.1}
0.943578 (0.000547) with: {'batch_size': 50, 'epochs': 100, 'model__optimizer': 'RMSprop', 'optimizer__learning_rate': 0.001}
0.935415 (0.008036) with: {'batch_size': 50, 'epochs': 100, 'model__optimizer': 'RMSprop', 'optimizer__learning_rate': 0.01}
0.942265 (0.013248) with: {'batch_size': 50, 'epochs': 100, 'model__optimizer': 'RMSprop', 'optimizer__learning_rate': 0.1}
0.370702 (0.082790) with: {'batch_size': 50, 'epochs': 100, 'model__optimizer': 'Adagrad', 'optimizer__learning_rate': 0.001}
0.425620 (0.094036) with: {'batch_size': 50, 'epochs': 100, 'model__optimizer': 'Adagrad', 'optimizer__learning_rate': 0.01}
0.410872 (0.048091) with: {'batch_size': 50, 'epochs': 100, 'model__optimizer': 'Adagrad', 'optimizer__learning_rate': 0.1}
-1.360142 (0.070818) with: {'batch_size': 50, 'epochs': 100, 'model__optimizer': 'Adadelta', 'optimizer__learning_rate': 0.001}
-1.

### Hyperparameter tuning - batch size, epoch, learning rate

In [10]:
# Function to create model, required for KerasClassifier
def create_model():
 # create model
 model = Sequential()
 model.add(Dense(units=200, input_shape=(X_train.shape[1],), activation='relu'))
 model.add(Dense(units=100, activation='relu'))
 model.add(Dense(units=50, activation='relu'))
 model.add(Dense(units=1, activation='linear'))
 return model

In [ ]:
# create model
model = KerasRegressor(model=create_model, verbose=0, optimizer = 'Nadam', loss = 'mean_squared_error', metrics = ['mae'])

# define the grid search parameters
batch_size = [50, 60, 80, 100]
epochs = [50, 100, 200]
learn_rate = [0.001, 0.01, 0.1]
#momentum = [0.8, 0.9]

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.997277 using {'batch_size': 60, 'epochs': 100, 'optimizer__learning_rate': 0.01}
0.976974 (0.017614) with: {'batch_size': 50, 'epochs': 50, 'optimizer__learning_rate': 0.01}
0.934511 (0.048929) with: {'batch_size': 50, 'epochs': 50, 'optimizer__learning_rate': 0.1}
0.289896 (0.414893) with: {'batch_size': 50, 'epochs': 50, 'optimizer__learning_rate': 0.2}
0.996621 (0.002431) with: {'batch_size': 50, 'epochs': 100, 'optimizer__learning_rate': 0.01}
0.880614 (0.097619) with: {'batch_size': 50, 'epochs': 100, 'optimizer__learning_rate': 0.1}
0.299880 (0.426459) with: {'batch_size': 50, 'epochs': 100, 'optimizer__learning_rate': 0.2}
0.992978 (0.004278) with: {'batch_size': 60, 'epochs': 50, 'optimizer__learning_rate': 0.01}
0.970004 (0.009733) with: {'batch_size': 60, 'epochs': 50, 'optimizer__learning_rate': 0.1}
0.298315 (0.438947) with: {'batch_size': 60, 'epochs': 50, 'optimizer__learning_rate': 0.2}
0.997277 (0.001802) with: {'batch_size': 60, 'epochs': 100, 'optimizer__learn

### Hyperparameter tuning - layers, neurons, activation function

In [10]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor as KR
import math
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [11]:
from tensorflow.keras.optimizers import RMSprop
def create_model(n_layers, first_layer_nodes, last_layer_nodes, activation_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(units = first_layer_nodes,  input_shape=(X_train.shape[1],), activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation='linear'))
    opt = RMSprop(learning_rate=0.1)
    model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['mae'])
    return model

##Wrap model into scikit-learn
model = KR(build_fn=create_model, verbose=0, epochs = 400, batch_size = 50)

C:\Users\Chathura Gamage\AppData\Local\Temp\ipykernel_10148\3417958943.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KR(build_fn=create_model, verbose=0, epochs = 400, batch_size = 50)


In [12]:
activation_funcs = ['relu', 'softplus', 'leaky_relu'] 
param_grid = dict(n_layers=[2,3], first_layer_nodes = [200,300], last_layer_nodes = [50],  activation_func = activation_funcs)
grid = GridSearchCV(estimator = model, param_grid = param_grid,  n_jobs=-1, cv=3, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

ERROR! Session/line number was not unique in database. History logging moved to new session 676
Best: 0.986263 using {'activation_func': 'softplus', 'first_layer_nodes': 200, 'last_layer_nodes': 50, 'n_layers': 2}
0.980573 (0.010610) with: {'activation_func': 'relu', 'first_layer_nodes': 200, 'last_layer_nodes': 50, 'n_layers': 2}
0.917243 (0.035116) with: {'activation_func': 'relu', 'first_layer_nodes': 200, 'last_layer_nodes': 50, 'n_layers': 3}
0.953663 (0.022865) with: {'activation_func': 'relu', 'first_layer_nodes': 300, 'last_layer_nodes': 50, 'n_layers': 2}
0.916777 (0.034610) with: {'activation_func': 'relu', 'first_layer_nodes': 300, 'last_layer_nodes': 50, 'n_layers': 3}
0.986263 (0.002684) with: {'activation_func': 'softplus', 'first_layer_nodes': 200, 'last_layer_nodes': 50, 'n_layers': 2}
0.958869 (0.009682) with: {'activation_func': 'softplus', 'first_layer_nodes': 200, 'last_layer_nodes': 50, 'n_layers': 3}
0.910386 (0.087254) with: {'activation_func': 'softplus', 'first